# Laboratorio#2: Modelos de recuperación de información

Un Modelo de Recuperación de Información (MRI) es un sistema diseñado para facilitar la búsqueda y extracción de información relevante sobre grandes bases de datos, por lo general estos datos se asocian a documentos pero pueden trabajar también con imágenes, audios o videos, o sea, con archivos de multimedia. Estos modelos son cruciales en áreas como los motores de búsqueda en internet, los sistemas de gestión documental y las bibliotecas digitales. Permiten a los usuarios encontrar información específica basada en criterios de búsqueda variados. Algunos de los modelos más significativos incluyen el modelo booleano, que utiliza operadores lógicos para la búsqueda; y el modelo vectorial, que emplea conceptos de álgebra vectorial para relacionar los datos con la consulta. 

Sin pérdida de generalidad, esta clase se centrará en los documentos como datos a recuperar. 

Formalmente, un MRI es un cuádruplo [D, Q, F, R(aj, di)] donde:
* D es un conjunto de representaciones lógicas de los documentos de la colección.
* Q es un conjunto compuesto por representaciones lógicas de las necesidades del usuario. Estas representaciones son denominadas consultas.
* F es un framework para modelar las representaciones de los documentos, consultas y sus relaciones.
* R es una función de ranking que asocia un número real con una consulta q perteneciente a Q y una representación de documento d perteneciente a D. La evaluación de esta función establece un cierto orden entre los documentos de acuerdo a la consulta.

## Modelo Booleano en la Recuperación de Información

El Modelo Booleano se basa en la teoría de conjuntos y el álgebra booleana, proporcionando un marco lógico y matemático para el proceso de búsqueda.

### Características Principales

- **Presencia de Términos:** Este modelo considera únicamente la presencia o ausencia de términos indexados en los documentos. No se toma en cuenta la frecuencia o el contexto de estos términos.

- **Binario en la Indexación:** Los términos indexados se asignan pesos binarios, Wi,j, que pueden ser 0 o 1. Estos representan la ausencia o presencia del término en el documento, respectivamente.

- **Estructura de Consulta:** Las consultas se formulan mediante términos indexados, los cuales están interconectados por tres operadores lógicos fundamentales: NOT, AND, OR. Esto permite realizar búsquedas complejas y específicas.

- **Expresión Lógica de las Consultas:** Las consultas pueden ser formuladas como una disyunción de conjunciones, conocida como Forma Normal Disyuntiva. Esto permite una mayor flexibilidad y precisión en la definición de los criterios de búsqueda.

## Configurando el entorno

In [ ]:
!pip3.10 install sympy
!python3.10 -m spacy download en_core_web_sm

In [2]:
import gensim
import spacy
from sympy import sympify, to_dnf, Not, And, Or

nlp = spacy.load("en_core_web_sm")

In [44]:
l = []
l.append(4)
l

[4]

Antes de comenzar con el desarollo de MRI, es importante pre-procesar el corpus. Este proceso puede tomar un tiempo considerable, por lo que se sugiere que al culminarlo almacene los resultados en memoria. Para facilitar tal proceso, se recomienda volcar la informacieon en una estructura cómoda: diccionario. Por supuesto, la información que aparezca en el diccionario es la deseada por cada equipo según sus consideraciones. 

A continuación se brinda funciones auxiliares necesarias.

In [39]:
import json

def save_json(file_path, data):
    with open(file_path, "w") as archivo_json:
        json.dump(mi_diccionario, archivo_json, indent=4)

def load_json(file_path): 
    with open(file_path, 'rb') as fp:
        data = json.load(fp)
    return data

def get_tokens(text):
    for token in nlp(text):
        if token.is_punct:
            pass
        else:
            yield token.lemma_


## Cargando el corpus tokenizado

In [40]:
tokenized_docs = []     # cargar del json

## Representación vectorial de los documentos

In [41]:
dictionary = gensim.corpora.Dictionary(tokenized_docs)
vocabulary = list(dictionary.token2id.keys())
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

## Representación de la consulta en FND

In [42]:
def query_to_dnf(query):
    query = query.lower().replace("and", "&").replace("or", "|").replace("not", "~")

    # Procesar la consulta con spaCy
    doc = nlp(query)

    # Filtrar y procesar tokens
    tokens = []
    for token in doc:
        # Filtrar manteniendo caracteres alfabéticos y símbolos específicos
        if token.text.isalpha() or token.text in ['&', '~', '|', '(', ')']:
            tokens.append(token.text)

    # Añadir '&' donde sea necesario
    processed_tokens = []
    for i in range(len(tokens)):
        processed_tokens.append(tokens[i])
        if i < len(tokens) - 1 and tokens[i] not in ['(', '&', '~', '|'] and tokens[i+1] not in [')', '&', '~', '|']:
            processed_tokens.append('&')

    # Unir la cadena
    processed_query = ' '.join(processed_tokens)

    # Convertir a expresión sympy y aplicar to_dnf
    query_expr = sympify(processed_query, evaluate=False)
    query_dnf = to_dnf(query_expr, simplify=True)

    return query_dnf

consulta = "A AND (B OR NOT C)"
consulta_dnf = query_to_dnf(consulta)
print(consulta_dnf)

(a & b) | (a & ~c)


## Definición de la función de peso

En el marco del modelo booleano, los pesos de los términos indexados se definen de manera binaria, denotados como `W_ij ∈ {0, 1}`. Aquí, una consulta `q` se representa como una expresión booleana convencional. Consideremos `q_fnd` como la Forma Normal Disyuntiva (FND) de la consulta `q`, y `q_ce` como una de las componentes conjuntivas de `q_fnd`.

La similitud entre un documento `d_i` y la consulta `q` se define de la siguiente manera:

- `sim(d_i, q) = 1` si existe al menos una componente conjuntiva `q_ce` en `q_fnd` tal que para todo término `k` en `q_ce`, la presencia del término en `d_i` es igual a la presencia del término en `q_ce`.
- En caso contrario, `sim(d_i, q) = 0`.

Esta definición establece una forma clara de medir la similitud entre documentos y consultas en el modelo booleano, basándose en la presencia o ausencia de términos específicos.


In [43]:
def get_matching_docs(query_dnf):
    global tokenized_docs, dictionary, corpus

    # Función para verificar si un documento satisface una componente conjuntiva de la consulta
    def satisfies_conjunctive_component(doc_bow, conjunctive_component):
        doc_terms = set(dict(doc_bow).keys())
        for term in conjunctive_component.args:
            term_id = dictionary.token2id.get(term.name, -1)
            if isinstance(term, Not):
                # Si el término está negado, debe estar ausente en el documento
                if term_id in doc_terms:
                    return False
            else:
                # Si el término no está negado, debe estar presente en el documento
                if term_id not in doc_terms:
                    return False
        return True
    
    matching_documents = []
    for doc_id, doc_bow in enumerate(corpus):
        # Revisar cada componente conjuntiva de la consulta DNF
        if isinstance(query_dnf, And) or isinstance(query_dnf, Or):
            # Si la consulta DNF tiene múltiples términos
            if any(satisfies_conjunctive_component(doc_bow, component) for component in query_dnf.args):
                matching_documents.append(doc_id)
        else:
            # Si la consulta DNF es un solo término o una única conjunción negada
            if satisfies_conjunctive_component(doc_bow, query_dnf):
                matching_documents.append(doc_id)

    return matching_documents

## Ejemplo

In [ ]:
query = ""     # insertar consulta
get_matching_docs(query_to_dnf(query))

## Otros modelos

1. **Modelo Booleano Difuso**: Es una variante del modelo booleano tradicional que introduce grados de pertenencia para los términos en los documentos. En lugar de utilizar valores binarios (0 o 1) para indicar la presencia o ausencia de términos, este modelo asigna valores en el rango de 0 a 1, representando el grado de relevancia o coincidencia de un término en un documento. Esto permite una recuperación más matizada y flexible que el modelo booleano estándar.

2. **Modelo Booleano Extendido**: Este modelo expande el modelo booleano clásico al permitir el uso de pesos en los términos de las consultas y en los documentos. Los pesos indican la importancia de un término dentro de una consulta o un documento, permitiendo una recuperación de información más precisa y flexible que el modelo booleano original, que solo considera la presencia o ausencia de términos.

3. **Modelo Vector Generalizado**: Es una extensión del modelo vectorial estándar que permite el uso de diferentes funciones de ponderación y similitud. En lugar de limitarse a medidas de similitud coseno típicas del modelo vectorial, el modelo vector generalizado puede incorporar diversas métricas para calcular la similitud entre documentos y consultas, ofreciendo así una mayor flexibilidad y potencialmente una mejora en la precisión de los resultados.

4. **Modelo de Indexación de Semántica Latente (LSI, por sus siglas en inglés)**: Este modelo se centra en capturar la relación semántica subyacente entre términos y documentos. Utiliza técnicas de descomposición matricial, como la descomposición en valores singulares, para reducir la dimensionalidad de la matriz término-documento. Al hacerlo, el LSI puede identificar patrones y asociaciones implícitas en el uso de palabras, mejorando la recuperación de información al superar algunas de las limitaciones del análisis de términos puramente sintáctico.